In [225]:
import re

In [226]:
MAP_FILE_LOCATION = "F:\\Projects\\MicroController\\STM32F446RE\\sotom_v1\\build\\final.map"
MAP_FILE = open(MAP_FILE_LOCATION)
MAP_LINES = MAP_FILE.readlines()
MAP_LINE = ""
MAP_DICT = {}

In [227]:
def isSection(line):
    if len(line) != 0 and line.startswith('.'):
        return True
    else:
        return False


def isSubSection(line):
    if len(line) != 0 and line.startswith(' ') and line.lstrip().startswith('.'):
        return True
    else:
        return False


def isValueSection(line):
    if len(line) != 0 and line.startswith('  ') and (not line.lstrip().startswith('.') or not line.lstrip().startswith('*')):
        return True
    else:
        return False


def addSectionToDict(dic, words, _stack, _subStack):
    key = words[0].removeprefix('.')
    key = key.replace('.', '__')
    print('SEC:'+key)
    print(dic)
    print(_stack)

    if len(_stack) != 0:
        dic[_stack[-1]]['is_closed'] = True
        if len(_subStack) != 0:
            _subStack.pop()
        _stack.pop()
    if dic != None and not key in dic.keys():
        dic[key] = {}
        _stack.append(key)
        if len(words) >= 3:
            dic[key]['origin'] = int(words[1], 16)
            dic[key]['size'] = int(words[2], 16)


def addSubSectionToSection(dic, words, _stack, section):
    key = words[0].removeprefix('.')
    key = key.replace('.', '__')

    print('SUB:'+key)
    print(dic)
    print(_stack)

    if len(_stack) != 0:
        dic[_stack[-1]]['is_closed'] = True
        _stack.pop()

    if dic != None and not key in dic:
        dic[key] = {}
        _stack.append(key)

    if len(words) >= 4:
        dic[key]['origin'] = int(words[1], 16)
        dic[key]['size'] = int(words[2], 16)
        dic[key]['file'] = words[3]

def addValue(dic,words,sectionStack,subsectionStack):
    words[1]=words[1].removeprefix('.')
    words[1]=words[1].removeprefix('__')
    words[1]=words[1].removeprefix('_')
    try:
        if len(subsectionStack)!=0 and len(words)==2:
            if not 'variables' in dic[sectionStack[-1]][subsectionStack[-1]]:
                dic[sectionStack[-1]][subsectionStack[-1]]['variables'] = {}
                dic[sectionStack[-1]][subsectionStack[-1]]['variables'][words[1]] = {}
            dic[sectionStack[-1]][subsectionStack[-1]]['variables'][words[1]]['origin']=int(words[0],16)
            dic[sectionStack[-1]][subsectionStack[-1]]['variables'][words[1]]['file'] = dic[sectionStack[-1]][subsectionStack[-1]]['file']
        elif len(words)>=3 and len(sectionStack)!=0:
            symName = ''
            symVal = 0
            if not 'symbols' in dic[sectionStack[-1]]:
                dic[sectionStack[-1]]['symbols']={}
                if len(words)>=4 and  words[3] == '.':
                    symName = words[1]
                    symVal = int(words[0],16)
                elif words[1] != '.':
                    symName = words[1]
                    symVal = int(words[0],16)
                dic[sectionStack[-1]]['symbols'][symName]={}
            dic[sectionStack[-1]]['symbols'][symName]['value']=symVal
    except KeyError as e:
        print("Key error {}".format(e))

In [228]:
HELPER_STACK_SECTION = []
HELPER_STACK_SUB_SECTION = []
for MAP_LINE in MAP_LINES:
    
    LINE_SPLIT = MAP_LINE.split()
    if len(LINE_SPLIT) == 0:
        continue
    
    if isSection(MAP_LINE):
        addSectionToDict(MAP_DICT,LINE_SPLIT,HELPER_STACK_SECTION,HELPER_STACK_SUB_SECTION)

    if isSubSection(MAP_LINE) and 1:
        addSubSectionToSection(MAP_DICT[HELPER_STACK_SECTION[-1]],LINE_SPLIT,HELPER_STACK_SUB_SECTION,HELPER_STACK_SECTION[-1])

    if isValueSection(MAP_LINE):
        addValue(MAP_DICT,LINE_SPLIT,HELPER_STACK_SECTION,HELPER_STACK_SUB_SECTION)


MAP_DICT

SEC:text
{}
[]
SUB:isr_vector
{'origin': 134217728, 'size': 4876, 'symbols': {'stext': {'value': 134217728}}}
[]
SUB:text
{'origin': 134217728, 'size': 4876, 'symbols': {'stext': {'value': 134217728}}, 'isr_vector': {'origin': 134217728, 'size': 452, 'file': 'build/startup_stm32f446xx.o', 'variables': {'vectors': {'origin': 134217728, 'file': 'build/startup_stm32f446xx.o'}}}}
['isr_vector']
Key error 'digitalWriteDebugButton'
Key error 'digitalReadDebugButton'
Key error 'alalogReadDebug'
Key error 'analogWriteDebug'
SUB:text
{'origin': 134217728, 'size': 4876, 'symbols': {'stext': {'value': 134217728}}, 'isr_vector': {'origin': 134217728, 'size': 452, 'file': 'build/startup_stm32f446xx.o', 'variables': {'vectors': {'origin': 134217728, 'file': 'build/startup_stm32f446xx.o'}}, 'is_closed': True}, 'text': {'origin': 134218180, 'size': 260, 'file': 'build/debug.o', 'variables': {'debugRamUsage': {'origin': 134218180, 'file': 'build/debug.o'}}}}
['text']
SUB:text
{'origin': 134217728, 'siz

{'text': {'origin': 134217728,
  'size': 4876,
  'symbols': {'stext': {'value': 134217728}},
  'isr_vector': {'origin': 134217728,
   'size': 452,
   'file': 'build/startup_stm32f446xx.o',
   'variables': {'vectors': {'origin': 134217728,
     'file': 'build/startup_stm32f446xx.o'}},
   'is_closed': True},
  'text': {'origin': 134221880,
   'size': 512,
   'file': 'build/usart.o',
   'variables': {'debugRamUsage': {'origin': 134218180,
     'file': 'build/debug.o'}},
   'is_closed': True},
  'rodata': {'origin': 134222544,
   'size': 60,
   'file': 'build/usart.o',
   'is_closed': True},
  'is_closed': True},
 'glue_7': {'origin': 134222604,
  'size': 0,
  'glue_7': {'origin': 134222604, 'size': 0, 'file': 'linker'},
  'is_closed': True},
 'glue_7t': {'origin': 134222604,
  'size': 0,
  'glue_7t': {'origin': 134222604, 'size': 0, 'file': 'linker'},
  'is_closed': True},
 'vfp11_veneer': {'origin': 134222604,
  'size': 0,
  'vfp11_veneer': {'origin': 134222604, 'size': 0, 'file': 'linke